# Elementary functions

Using the interactive system, let us define the square function and the recursive factorial function. Then, let us apply these functions to sample values:

In [15]:
let square x = x * x;;
let rec fact x =
  if x <= 1 then 1 else x * fact (x - 1);;

val square : int -> int = <fun>
val fact : int -> int = <fun>


In [5]:
fact 5;;
square 120;;

- : int = 120
- : int = 14400


# Automatic memory management

All allocation and deallocation operations are fully automatic. For example, let us consider simply linked lists.

Lists are predefined in Caml. The empty list is written []. The constructor that allows prepending an element to a list is written :: (in infix form).

In [6]:
let l = 1 :: 2 :: 3 :: [];;

val l : int list = [1; 2; 3]


In [7]:
[1; 2; 3];;

- : int list = [1; 2; 3]


In [8]:
5 :: l;;

- : int list = [5; 1; 2; 3]


# Polymorphism: sorting lists

Insertion sort is defined using two recursive functions.

In [3]:
let rec sort = function
  | [] -> []
  | x :: l -> insert x (sort l)
and insert elem = function
  | [] -> [elem]
  | x :: l -> 
      if elem < x then elem :: x :: l else x :: insert elem l;;

val sort : 'a list -> 'a list = <fun>
val insert : 'a -> 'a list -> 'a list = <fun>


Note that the type of the list elements remains unspecified: it is represented by a *type variable* `'a`. Thus, sort can be applied both to a list of integers and to a list of strings.

In [4]:
sort [2; 1; 0];;
sort ["yes"; "ok"; "sure"; "ya"; "yep"];;

- : int list = [0; 1; 2]
- : string list = ["ok"; "sure"; "ya"; "yep"; "yes"]


# Imperative features

Let us encode polynomials as arrays of integer coefficients. Then, to add two polynomials, we first allocate the result array, then fill its slots using two successive `for` loops.

In [5]:
let add_polynom p1 p2 =
  let n1 = Array.length p1
  and n2 = Array.length p2 in
  let result = Array.create (max n1 n2) 0 in
  for i = 0 to n1 - 1 do result.(i) <- p1.(i) done;
  for i = 0 to n2 - 1 do result.(i) <- result.(i) + p2.(i) done;
  result;;

val add_polynom : int array -> int array -> int array = <fun>


In [6]:
add_polynom [| 1; 2 |] [| 1; 2; 3 |];;

- : int array = [|2; 4; 3|]


Caml offers updatable memory cells, called *references*: `ref init` returns a new cell with initial contents `init`, `!cell` returns the current contents of cell, and cell `:=` v writes the value v into cell.

We may redefine fact using a reference cell and a for loop:

In [7]:
let fact n =
    let result = ref 1 in
    for i = 2 to n do result := i * !result done;
    !result;;

val fact : int -> int = <fun>


# Higher-order functions

There is no restriction on functions, which may thus be passed as arguments to other functions. Let us define a function sigma that returns the sum of the results of applying a given function f to each element of a list:



In [8]:
let rec sigma f = function
    | [] -> 0
    | x :: l -> f x + sigma f l;;

val sigma : ('a -> int) -> 'a list -> int = <fun>


Anonymous functions may be defined using the function construct:

In [9]:
sigma (function x -> x * x) [1; 2; 3];;

- : int = 14


Polymorphism and higher-order functions allow defining function composition in its most general form:

In [17]:
let compose f g = (function x -> f (g x));;
let square_o_fact = compose square fact;;
square_o_fact 5;;

val compose : ('a -> 'b) -> ('c -> 'a) -> 'c -> 'b = <fun>
val square_o_fact : int -> int = <fun>
- : int = 14400


# The power of functions

The power of functions cannot be better illustrated than by the power function:

In [21]:
let rec power f n = 
    if n = 0 then function x -> x 
    else compose f (power f (n - 1));;

val power : ('a -> 'a) -> int -> 'a -> 'a = <fun>


In [22]:
let derivative dx f = function x -> (f (x +. dx) -. f x) /. dx;;
let sin''' = power (derivative 1e-5) 3 sin;;
let pi = 4.0 *. atan 1.0 in sin''' pi;;

val derivative : float -> (float -> float) -> float -> float = <fun>
val sin''' : float -> float = <fun>
- : float = 0.999998972517346263


# Symbolic computation

Let us consider simple symbolic expressions made up of integers, variables, let bindings, and binary operators. Such expressions can be defined as a new data type, as follows:

In [18]:
type expression =
  | Num of int
  | Var of string
  | Let of string * expression * expression
  | Binop of string * expression * expression;;

type expression =
    Num of int
  | Var of string
  | Let of string * expression * expression
  | Binop of string * expression * expression


Evaluation of these expressions involves an environment that maps identifiers to values, represented as a list of pairs.

In [19]:
let rec eval env = function
  | Num i -> i
  | Var x -> List.assoc x env
  | Let (x, e1, in_e2) ->
     let val_x = eval env e1 in
     eval ((x, val_x) :: env) in_e2
  | Binop (op, e1, e2) ->
     let v1 = eval env e1 in
     let v2 = eval env e2 in
     eval_op op v1 v2

and eval_op op v1 v2 =
  match op with
  | "+" -> v1 + v2
  | "-" -> v1 - v2
  | "*" -> v1 * v2
  | "/" -> v1 / v2
  | _ -> failwith ("Unknown operator: " ^ op);;

val eval : (string * int) list -> expression -> int = <fun>
val eval_op : string -> int -> int -> int = <fun>


As an example, we evaluate the phrase `let x = 1 in x + x`:

In [20]:
eval [] (Let ("x", Num 1, Binop ("+", Var "x", Var "x")));;

- : int = 2
